<a href="https://colab.research.google.com/github/DemirDorukDilek/CBS/blob/main/enson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
import numpy as np
import networkx as nx
import heapq
from itertools import product
import os
from graphviz import Digraph
import uuid

data = "./mapcik.txt"
if not os.path.isfile(data):
    mapcik = [
    (1, 2),
    (1, 4),
    (2, 1),
    (2, 5),
    (3, 4),
    (3, 7),
    (4, 1),
    (4, 3),
    (4, 5),
    (4, 8),
    (5, 2),
    (5, 4),
    (5, 6),
    (5, 9),
    (6, 5),
    (6, 10),
    (7, 3),
    (7, 8),
    (8, 4),
    (8, 7),
    (8, 9),
    (8, 11),
    (9, 5),
    (9, 8),
    (9, 10),
    (9, 12),
    (10, 6),
    (10, 9),
    (11, 8),
    (11, 12),
    (12, 9),
    (12, 11),
],(1,3),(12,10)
    mapcik2 = [
    (10,11),
    (10,12),
    (11,30),
    (12,30),
    (20,21),
    (20,22),
    (21,30),
    (22,30),
    (30,15),
    (30,25)
],(10,20),(15,25)
    mapcik3 = [
        (1,2),
        (1,3),
        (2,1),
        (3,2)
    ], (1,2),(2,1)
    mapcik21 = [
    (10,11),
    (10,12),
    (11,50),
    (12,50),
    (20,21),
    (20,22),
    (21,50),
    (22,50),
    (50,15),
    (50,25),
    (30,31),
    (30,32),
    (31,50),
    (32,50),
    (50,35),
],(10,20,30),(15,25,35)
    mapa,START,END = mapcik3
    net = nx.DiGraph()
    net.add_edges_from(mapa)
else:
    net = nx.read_edgelist("./mapcik.txt", edgetype=int, nodetype=int)

In [140]:

class Anode:
    __slots__ = ('state','f','g','parent','time')
    def __init__(self,state,f=None,g=0,parent=None,time=0):
        self.state=state
        self.f=f
        self.g = g
        self.parent=parent
        self.time=time
    def __lt__(self,o):
        return self.f<o.f
    def __repr__(self):
        return str(self.state)

def get_successor(state,adj):
    return product(*((u,)+tuple(adj[u]) for u in state))

def MAPF_valid(os,ns):
    return len(os) == len(set(ns)) and len(os) == len({frozenset(x) for x in zip(os,ns)})

def is_goal(state,goal):
    return state == goal

def get_path(node):
    vertexlist = [node.state]
    node = node.parent
    edgelist = []
    while node:
        edgelist.append(frozenset((node.state,vertexlist[-1])))
        vertexlist.append(node.state)
        node = node.parent
    vertexlist.reverse()
    edgelist.reverse()
    return vertexlist,edgelist

def reverse_dijkstra(graph, goals):
    rg = graph.reverse()
    h_values = []
    for goal in goals:
        h_values.append(nx.single_source_shortest_path_length(rg,goal))

    return h_values
def SIC(state,h_values):
    return sum([h_values[aidx][x] if x in h_values[aidx] else float("inf") for aidx,x in enumerate(state)])

def astar(G,start=(1,3),goal=(12,10),heuristic=None,heuristic_precalculator=None,constraint=None,agent=None):
    # Local variable cache
    heappush = heapq.heappush
    heappop = heapq.heappop
    adj = G.adj

    OPEN = []
    CLOSED = {}

    if heuristic_precalculator:
        h_values = heuristic_precalculator(G,goal)

    node = Anode(start,heuristic(start,h_values),0,None)
    OPENd = {(start,node.time):node.f}
    heappush(OPEN,node)

    while OPEN:
        q = heappop(OPEN)
        OPENd.pop((q.state,q.time))

        if is_goal(q.state,goal):return get_path(q)
        if (q.state,q.time) in CLOSED: continue
        CLOSED[(q.state,q.time)] = q.f
        for s in get_successor(q.state,adj):
            if not MAPF_valid(q.state,s): continue
            if (agent,s,q.time+1) in constraint: continue
            if (agent,frozenset((q.state,s)),q.time+1) in constraint: continue

            g = q.g+1
            h = heuristic(s,h_values)
            f = g+h

            n = Anode(s,f,g,q,q.time+1)
            if OPENd.get((n.state,n.time),float("inf")) > n.f:
                heappush(OPEN,n)
                OPENd[(n.state,n.time)] = n.f

In [145]:
class CTNode:
    def __init__(self,cost = None,con = None,vertexlist = None,edgelist = None, parent = None):
        self.cost = cost
        self.con = con
        self.vertexlist = vertexlist
        self.edgelist = edgelist
        self.parent = None
        self.children = []
        self.uuid = str(uuid.uuid1())

    def root(self):
        self.cost = 0
        self.con = set()
        self.vertexlist = []
        self.edgelist = []

    def branch(self,new_con):
        node = CTNode(self.cost,self.con.union(new_con),self.vertexlist.copy(),self.edgelist.copy(), self)
        return node

    def __repr__(self):
        temp = []
        for i in self.vertexlist:
            temp.append(str([x[0] if len(x) == 1 else x for x in i]))
        return str(self.con) + "\nCost:" + str(self.cost) + "\nSolutions:\n\t" + "\n\t".join(temp)

    def __str__(self):
        txt = ""
        idxs = []
        cc = f"    Cost:{self.cost}"
        for loop,i in enumerate(self.con or set(((0,(0,),0),))):
            if type(i[1]) == frozenset:
                x = (i[0],tuple(map(lambda x:x[0],i[1])),i[2])
            else:
                x = (i[0],i[1][0],i[2])
            txt += str(x)
            if not loop:
                txt += cc
            else:
                txt += " "*(len(cc)+4)
            txt += "\n"
            idxs.append(i[0])
        txt+="-------\n"
        for loop,i in enumerate(self.vertexlist):
            if loop in idxs or True:
                txt += str([x[0] if len(x) == 1 else x for x in i])
                txt += "\n"
        return txt


    def __lt__(self,o):
        return self.cost<o.cost

def check_conflit(solution):
    conflit = False
    for timestep,i in enumerate(zip(*solution)):
        count={}
        index={}
        for agent,j in enumerate(i):
            count[j] = count.get(j,0)+1
            index[j] = index.get(j,[])+[agent]
        for j in count:
            if count[j] > 1:
                conflit = (index[j],j,timestep)
        if conflit:
            break
    return conflit

def high_level(G,start,goal):
    tree = Digraph();tree.attr(rankdir="TB")

    OPEN = []
    root = CTNode()
    root.root()
    for i in range(len(start)):
        x = astar(G,(start[i],),(goal[i],),heuristic=SIC,heuristic_precalculator=reverse_dijkstra,constraint=set(),agent=i)
        if x:
            root.vertexlist.append(x[0])
            root.edgelist.append(x[1])
            root.cost += len(root.vertexlist[-1])-1
        else:
            return

    heapq.heappush(OPEN,root)
    tree.node(root.uuid,str(root),shape="box")

    while OPEN:
        input(":")
        # # print()
        # # print()
        # # print()
        # # print()
        # # print()
        # # for i in OPEN:
        # #     print()
        # #     print(i.__repr__())
        tree.render("tree",view=True,format="png",cleanup=True)


        q = heapq.heappop(OPEN)
        conflit = check_conflit(q.vertexlist)
        if not conflit:
            conflit = check_conflit(q.edgelist)
            if not conflit:
                return q.vertexlist,root

        new_solutions = {}
        constraints = {}
        for i in conflit[0]:
            constraints[i] = (i,conflit[1],conflit[2]+1 if type(conflit[1]) == frozenset else conflit[2])
            new_solutions[i] = astar(G,(start[i],),(goal[i],),heuristic=SIC,heuristic_precalculator=reverse_dijkstra,constraint=q.con.union(frozenset((constraints[i],))),agent=i)
        conset = set(constraints.values())
        for i in conflit[0]:
            kaput = False
            new = q.branch(conset.difference(frozenset((constraints[i],))))
            for j in new_solutions:
                if j != i:
                    if not new_solutions[j]:
                        kaput = True
                        break
                    new.vertexlist[j],new.edgelist[j] = new_solutions[j]
                    new.cost += len(new.vertexlist[j])-len(q.vertexlist[j])
            if not kaput:
                heapq.heappush(OPEN,new)
                q.children.append(new)
                tree.node(new.uuid,str(new),shape="box")
                tree.edge(q.uuid,new.uuid)




raw,root = high_level(net,start=START,goal=END)
temp = []
for i in raw:
    temp.append(str([x[0] if len(x) == 1 else x for x in i]))
print("Final Solution:",end="")
print("",*temp,sep="\n\t")


:



KeyboardInterrupt: Interrupted by user

In [142]:

dot = Digraph();dot.attr(rankdir="TB")
def preorder(node,parent=None):
    x = str(uuid.uuid1())
    dot.node(x,str(node),shape="box")
    if parent:dot.edge(parent,x)
    for i in node.children:preorder(i,x)
preorder(root);dot.render("akame",view=True,format="png",cleanup=True)


'akame.png'